# Hiking with Tobler: Tracking Movement and Calibrating a Cost Function for Personalized 3D Accessibility
author: Christopher D. Higgins  
email: cd.higgins@utoronto.ca  
affiliation: University of Toronto Scarborough

This notebook presents the code for running the network analysis component of the paper. The bundled geodatabase has building centroids with a unique `BLDG_ID` for the Central and Western District of Hong Kong, downloaded by way of the [iB1000 tiles](https://geodata.gov.hk/gs/view-dataset?uuid=14cc7b80-900f-42d5-a109-015b7ef6dcf6&sidx=0) produced by the Hong Kong Government's Lands Department. The remainder of the workbook downloads a district boundary file and the 3D pedestrian network, prepares the network, and calculates an origin-destination cost matrix from all buildings to the Kinwick Centre.

In [ ]:
# import packages
import os # for operating-system stuff like file paths
import arcpy # to run arcgis tools from python
import requests # to download files from the internet
import zipfile # to unzip files
import io # to write files to disk

# create a general downloader/unzipper function
def download_zip(url): # defines the function and parameters as function_name(parameters)
    package = requests.get(url) # uses 'get' from the requests package to get a file from a url
    zip_package = zipfile.ZipFile(io.BytesIO(package.content)) # this tells the ZipFile function that the content from our package is in zip format
    zip_package.extractall() # use extractall to unzip the contents of zip_package
    
# another helper function for downloading other file types like geojson
def download_file(url, file_name): 
    package = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(package.content)
    
# get environment variables
work_dir = os.getcwd()
work_gdb = arcpy.env.workspace

print(work_dir)
print(work_gdb)

## 1. Download and Import Files

In [ ]:
# download files
## boundary json
download_file(url = "https://www.had.gov.hk/psi/hong-kong-administrative-boundaries/hksar_18_district_boundary.json",
              file_name = "hksar_18_district_boundary.json")


## 3d pedestrian network
download_zip(url = "https://geodata.gov.hk/gs-data-files/201eaaee-47d6-42d0-ac81-19a430f63952/3DPN_GDB.zip")

In [ ]:
# import files
## create spatial reference for the boundary data
sr2d = "2326"

# create datasets in gdb
## boundary for boundary files
arcpy.management.CreateFeatureDataset(out_dataset_path = work_gdb,
                                      out_name = "boundary",
                                      spatial_reference = sr2d)

## boundary
arcpy.conversion.JSONToFeatures(in_json_file = os.path.join(work_dir, "hksar_18_district_boundary.json"), 
                                out_features = os.path.join(work_gdb, "boundary", "district_boundary"), 
                                geometry_type = "POLYGON")

# make it a layer
boundary = os.path.join(work_gdb, "boundary", "district_boundary")
arcpy.management.MakeFeatureLayer(in_features = boundary, out_layer = "boundary_lyr")

In [ ]:
# import and trim down pedestrian network to just central-western
pn3d = os.path.join(work_dir, "3DPN.gdb\PedestrianRoute")
arcpy.management.MakeFeatureLayer(in_features = pn3d, out_layer = "pn3d_lyr")

# get spatial reference for 3dpn
sr3d = arcpy.Describe(pn3d).spatialReference

## pn3d for pedestrian network
arcpy.management.CreateFeatureDataset(out_dataset_path = work_gdb,
                                      out_name = "pn3d",
                                      spatial_reference = sr3d)

# select network on hk island
arcpy.management.SelectLayerByAttribute(in_layer_or_view = "boundary_lyr", 
                                        selection_type = "NEW_SELECTION", 
                                        where_clause = "District = 'Central & Western' "+
                                                        "Or District = 'Wan Chai' "+
                                                        "Or District = 'Eastern' "+
                                                        "Or District = 'Southern'")

arcpy.management.SelectLayerByLocation(in_layer = "pn3d_lyr", 
                                       overlap_type = "INTERSECT", 
                                       select_features = "boundary_lyr", 
                                       search_distance = None, 
                                       selection_type = "NEW_SELECTION", 
                                       invert_spatial_relationship = "NOT_INVERT")

arcpy.conversion.FeatureClassToFeatureClass(in_features = "pn3d_lyr", 
                                            out_path = os.path.join(work_gdb), 
                                            out_name = "PedestrianRoute_Island")

pn3d_island = os.path.join(work_gdb, "PedestrianRoute_Island")
arcpy.management.MakeFeatureLayer(in_features = pn3d_island, out_layer = "pn3d_island_lyr")

## 2. Prepare Network Dataset

In [ ]:
# prepare network
## add fields
arcpy.management.AddField(pn3d_island, "MECH", "SHORT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

## select mtr links and delete
arcpy.management.SelectLayerByAttribute("pn3d_island_lyr", "NEW_SELECTION", "FeatureType >= 15 And FeatureType <= 22", None)
arcpy.management.DeleteRows(in_rows = "pn3d_island_lyr")

## select mechanized links and flag
arcpy.management.SelectLayerByAttribute("pn3d_island_lyr", "NEW_SELECTION", "FeatureType >= 8 And FeatureType <= 10", None)
arcpy.management.CalculateField("pn3d_island_lyr", "MECH", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute("pn3d_island_lyr", "SWITCH_SELECTION", '', None)
arcpy.management.CalculateField("pn3d_island_lyr", "MECH", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute("pn3d_island_lyr", "CLEAR_SELECTION", '', None)

In [ ]:
# split lines at points
split_dist = "10 Meters"
arcpy.management.GeneratePointsAlongLines(Input_Features = pn3d_island, 
                                          Output_Feature_Class = os.path.join(work_gdb, "PedestrianRoute_Island_Points"), 
                                          Point_Placement = "DISTANCE", 
                                          Distance = split_dist, 
                                          Percentage = None, 
                                          Include_End_Points = "NO_END_POINTS")

points = os.path.join(work_gdb, "PedestrianRoute_Island_Points")

arcpy.management.SplitLineAtPoint(in_features = pn3d_island, 
                                  point_features = points, 
                                  out_feature_class = os.path.join(work_gdb, "pn3d", "PedestrianRoute_Island_Split"), 
                                  search_radius = "0.001 Meters")

lines_split = os.path.join(work_gdb, "pn3d", "PedestrianRoute_Island_Split")

In [ ]:
# define tobler function
def tobler_calc(input_fc, alpha, beta1, beta2):
    # Add Z Information
    arcpy.management.AddField(input_fc, "Start_Z", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.AddField(input_fc, "End_Z", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.AddField(input_fc, "Length2D", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.CalculateField(input_fc, "Start_Z", "!SHAPE!.firstpoint.Z", "PYTHON3", "")
    arcpy.management.CalculateField(input_fc, "End_Z", "!SHAPE!.lastpoint.Z", "PYTHON3", "")
    arcpy.management.CalculateField(input_fc, "Length2D", "!SHAPE!.LENGTH", "PYTHON3", "")
    arcpy.ddd.AddZInformation(input_fc, "LENGTH_3D;AVG_SLOPE", "NO_FILTER")

    # Add walk time fields
    arcpy.management.AddField(input_fc, "FT_MIN_2D", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.AddField(input_fc, "TF_MIN_2D", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.AddField(input_fc, "FT_MIN_3D", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.AddField(input_fc, "TF_MIN_3D", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.AddField(input_fc, "FT_MIN_TB", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
    arcpy.management.AddField(input_fc, "TF_MIN_TB", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")

    # Calculate walk time fields
    fields = ["FT_MIN_2D", "TF_MIN_2D", "FT_MIN_3D", "TF_MIN_3D", 
              "Length2D", "Length3D", "Start_Z", "End_Z", "MECH",
              "FT_MIN_TB", "TF_MIN_TB"]
    with arcpy.da.UpdateCursor(input_fc, fields) as updateRows:
        for updateRow in updateRows:
            # non mechanical
            if updateRow[8] == 0:
                updateRow[0] = (updateRow[4]/((alpha*(math.exp((-beta1)*(math.fabs(0+beta2)))))*1000))*60 # flat ground speed
                updateRow[1] = (updateRow[4]/((alpha*(math.exp((-beta1)*(math.fabs(0+beta2)))))*1000))*60 # flat ground speed
                
                # cap gradient to ~100%
                if math.fabs((updateRow[7]-updateRow[6])/(updateRow[4])) <= 1:
                    updateRow[2] = (updateRow[5]/((alpha*(math.exp((-beta1)*(math.fabs((updateRow[7]-updateRow[6])/(updateRow[4])+beta2)))))*1000))*60
                    updateRow[3] = (updateRow[5]/((alpha*(math.exp((-beta1)*(math.fabs((updateRow[6]-updateRow[7])/(updateRow[4])+beta2)))))*1000))*60
                    updateRow[9] = (updateRow[5]/((6*(math.exp((-3.5)*(math.fabs((updateRow[7]-updateRow[6])/(updateRow[4])+0.05)))))*1000))*60
                    updateRow[10] = (updateRow[5]/((6*(math.exp((-3.5)*(math.fabs((updateRow[6]-updateRow[7])/(updateRow[4])+0.05)))))*1000))*60
                else:
                    updateRow[2] = (updateRow[5]/((alpha*(math.exp((-beta1)*(math.fabs(1+beta2)))))*1000))*60
                    updateRow[3] = (updateRow[5]/((alpha*(math.exp((-beta1)*(math.fabs(1+beta2)))))*1000))*60
                    updateRow[9] = (updateRow[5]/((alpha*(math.exp((-beta1)*(math.fabs(1+beta2)))))*1000))*60
                    updateRow[10] = (updateRow[5]/((alpha*(math.exp((-beta1)*(math.fabs(1+beta2)))))*1000))*60
                
                updateRows.updateRow(updateRow)
            # mechanlical like elevators and travelators
            else:
                updateRow[0] = (updateRow[5]/2000)*60
                updateRow[1] = (updateRow[5]/2000)*60
                updateRow[2] = (updateRow[5]/2000)*60
                updateRow[3] = (updateRow[5]/2000)*60
                updateRow[9] = (updateRow[5]/2000)*60
                updateRow[10] = (updateRow[5]/2000)*60
                updateRows.updateRow(updateRow)

In [ ]:
# calculate travel times
tobler_calc(input_fc = lines_split, alpha = 4.607, beta1 = 1.542, beta2 = 0.033)

In [ ]:
# create nd from template
arcpy.na.CreateNetworkDatasetFromTemplate(network_dataset_template = os.path.join(work_dir, "PedestrianRoute_Island_ND.xml"), 
                                          output_feature_dataset = os.path.join(work_gdb, "pn3d"))

# build network
pn3d_nd = os.path.join(work_gdb, "pn3d", "PedestrianRoute_Island_ND")
arcpy.na.BuildNetwork(pn3d_nd)

## 3. Perform Network Analysis

In [ ]:
# loop over modes
mode_list = ["Walk_2D","Walk_NLS","Walk_Tobler"]

for i in mode_list:
    mode_name = i
    odcm_name = "odcm"+i

    # create odcm layer
    arcpy.na.MakeODCostMatrixAnalysisLayer(network_data_source = pn3d_nd, 
                                           layer_name = odcm_name, 
                                           travel_mode = mode_name, 
                                           line_shape = "NO_LINES",
                                           accumulate_attributes = "Length;Walk_2D;Walk_NLS;Walk_Tobler")

    origins_i = os.path.join(work_gdb, "buildings", "BuildingCentroids_Central")
    destinations_j = os.path.join(work_gdb, "buildings", "KinwickCentroid")

    ## add origins
    arcpy.na.AddLocations(in_network_analysis_layer = odcm_name, 
                          sub_layer = "Origins", 
                          in_table = origins_i, 
                          field_mappings = "Name BLDG_ID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Length # #;Cutoff_Walk_NLS # #;Cutoff_Walk_Tobler # #;Cutoff_Walk_2D # #", 
                          search_tolerance = "5000 Meters", 
                          sort_field = None, 
                          search_criteria = "PedestrianRoute_Island_Split SHAPE;PedestrianRoute_Island_ND_Junctions NONE", 
                          match_type = "MATCH_TO_CLOSEST", 
                          append = "CLEAR", 
                          snap_to_position_along_network = "NO_SNAP")

    arcpy.na.AddLocations(in_network_analysis_layer = odcm_name, 
                          sub_layer = "Destinations", 
                          in_table = destinations_j, 
                          field_mappings = "Name BLDG_ID #;CurbApproach # 0", 
                          search_tolerance = "5000 Meters", 
                          sort_field = None, 
                          search_criteria = "PedestrianRoute_Island_Split SHAPE;PedestrianRoute_Island_ND_Junctions NONE", 
                          match_type = "MATCH_TO_CLOSEST", 
                          append = "CLEAR", 
                          snap_to_position_along_network = "NO_SNAP")

    ## solve
    arcpy.na.Solve(in_network_analysis_layer = odcm_name)

    ## export travel times
    arcpy.conversion.TableToTable(in_rows = odcm_name+"\Lines", 
                                  out_path = work_dir, 
                                  out_name = mode_name+".csv")